In [ ]:
https://www.linkedin.com/pulse/hyper-parameter-optimisation-using-genetic-algorithms-conor-rothwell

In [1]:
import random
import numpy as np
import pandas as pd
import kf_ml_lib as kf
import matplotlib.pyplot as plt

from deap import base
from deap import creator
from deap import tools
from deap import algorithms

from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn.model_selection import train_test_split


def mutate(individual):
    gene = random.randint(0,9) #select which parameter to mutate
    if gene == 0:
        if individual[0] == 'gini':
            individual[0] = 'entropy'
        else:
            individual[0] = 'gini'
        
    elif gene == 1:
        if individual[1] == 'best':
            individual[1] = 'random'
        else:
            individual[1] = 'best'
    
    elif gene == 2:
        individual[2] = random.randint(lower_depth, upper_depth)
        
    elif gene in [4,5]:
        individual[gene] = random.uniform(lower_samples_leaf, upper_samples_leaf)
        
    elif gene in [3,6,8]:
        individual[gene] = random.random()
        
    elif gene == 7:
        individual[7] = random.randint(lower_leaf_nodes, upper_leaf_nodes)
        
    elif gene == 9:
        if individual[9] == 'balanced':
            individual[9] = None
        else:
            individual[9] = 'balanced'        
        
    return individual,


def evaluate(individual):
    #'''
    # build and test a model based on the parameters in an individual and return
    # the AUROC value
    #'''
    # extract the values of the parameters from the individual chromosome
    crit = individual[0]
    split = individual[1]
    depth = individual[2]
    samples_split = individual[3]
    samples_leaf = individual[4]
    weight_fraction_leaf = individual[5]
    features = individual[6]
    leaf_nodes = individual[7]
    impurity_decrease = individual[8]
    class_weight = individual[9]
    
    # build the model
    model = DecisionTreeClassifier(criterion=crit, 
                                   splitter=split, 
                                   max_depth=depth, 
                                   min_samples_split=samples_split, 
                                   min_samples_leaf=samples_leaf, 
                                   min_weight_fraction_leaf=weight_fraction_leaf, 
                                   max_features=features, 
                                   max_leaf_nodes=leaf_nodes, 
                                   min_impurity_decrease=impurity_decrease, 
                                   class_weight=class_weight)

    model = model.fit(X_train, y_train)
    
    # Make predictions on the test data
    preds = model.predict(X_test)
    
    # Evaluate model Precision, Recall and F1-Score performance
    precision = metrics.precision_score(y_test, preds, pos_label='Botnet')
    recall = metrics.recall_score(y_test, preds, pos_label='Botnet')
    f1_score = kf.calc_f1_score(precision, recall)
    
    print("f1_score = ", f1_score)
    
    return f1_score,



creator.create("FitnessMax", base.Fitness, weights=(1.0,)) # Maximise the fitness function value
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()

# Possible parameter values
criterions = ['gini', 'entropy']
splits = ['best', 'random']
lower_depth, upper_depth = 2, 100 
lower_leaf_nodes, upper_leaf_nodes = 2, 100
class_weight = ['balanced', None]
lower_samples_leaf, upper_samples_leaf  = 0, 0.5


N_CYCLES = 1

# Init chromosomes; depending upon their possible parameter values or ranges
toolbox.register("attr_crit", random.choice, criterions)
toolbox.register("attr_splitter", random.choice, splits)
toolbox.register("attr_depth", random.randint, lower_depth, upper_depth)
toolbox.register("attr_flt", random.random)
toolbox.register("attr_leaf_nodes", random.randint, lower_leaf_nodes, upper_leaf_nodes)
toolbox.register("attr_weight", random.choice, class_weight)
toolbox.register("attr_samples_leaf", random.uniform, lower_samples_leaf, upper_samples_leaf)

# Init chromosomes, with relevant randomly initalised genes
toolbox.register("individual", tools.initCycle, creator.Individual,
                 (toolbox.attr_crit, toolbox.attr_splitter, toolbox.attr_depth,
                 toolbox.attr_flt, toolbox.attr_samples_leaf, toolbox.attr_samples_leaf, 
                 toolbox.attr_flt, toolbox.attr_leaf_nodes,
                 toolbox.attr_flt, toolbox.attr_weight), n=N_CYCLES)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# Set GA functions
toolbox.register("mate", tools.cxOnePoint)
toolbox.register("mutate", mutate)
toolbox.register("select", tools.selTournament, tournsize=5, fit_attr='fitness')
toolbox.register("evaluate", evaluate)

population_size = 10
crossover_probability = 0.7
mutation_probability = 0.7
number_of_generations = 20

pop = toolbox.population(n=population_size)
hof = tools.HallOfFame(1)
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", np.mean)
stats.register("std", np.std)
stats.register("min", np.min)
stats.register("max", np.max)


# Load, process and split dataset
dataset = kf.load_dataset("../Datasets/ISOT Botnet 2010/Pre-processed/isot_botnet.csv")
X, y = kf.split_dataset(dataset, extended=False)

y = np.split(y, 1)
y = np.array(y)
y = y.reshape(len(y[0]), 1)
        
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, stratify=y)

# Run GA
pop, log = algorithms.eaSimple(pop, toolbox, cxpb=crossover_probability, stats = stats, 
                               mutpb = mutation_probability, ngen=number_of_generations, halloffame=hof, 
                               verbose=True) 

# Record best paramaters from the halloffame
best_parameters = hof[0]

print("BEST PARAMETERS = ", best_parameters)


# Process data for plotting
gen = log.select("gen")
max_ = log.select("max")
avg = log.select("avg")
min_ = log.select("min")

evolution = pd.DataFrame({'Generation': gen,
                         'Max F1-Score': max_,
                          'Average':avg,
                         'Min F1-Score': min_})

plt.title('Parameter Optimisation')
plt.plot(evolution['Generation'], evolution['Min F1-Score'], 'b', color = 'C1',
         label = 'Min F1-Score')
plt.plot(evolution['Generation'], evolution['Average'], 'b', color = 'C2',
         label = 'Average F1-Score')
plt.plot(evolution['Generation'], evolution['Max F1-Score'], 'b', color = 'C3',
         label= 'Max F1-Score')


plt.legend(loc = 'lower right')
plt.ylabel('F1-Score')
plt.xlabel('Generation')
plt.xticks([0,5,10,15,20])


f1_score =  0.5120032990662465


c:\users\asmarus\anaconda3\envs\tensorflow-gpu\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
G:\USB Drive Files\Year 3\Semester 1\KV6003 - Individual Computing Project\Machine Learning\kf_ml_lib.py:91: RuntimeWarning: invalid value encountered in double_scalars
  f1_score = 2 * ((precision*recall) / (precision+recall))


f1_score =  nan
f1_score =  0.6386312324891874
f1_score =  0.5120032990662465
f1_score =  0.5120032990662465


c:\users\asmarus\anaconda3\envs\tensorflow-gpu\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
G:\USB Drive Files\Year 3\Semester 1\KV6003 - Individual Computing Project\Machine Learning\kf_ml_lib.py:91: RuntimeWarning: invalid value encountered in double_scalars
  f1_score = 2 * ((precision*recall) / (precision+recall))


f1_score =  nan


...

f1_score =  nan
f1_score =  0.5120032990662465


c:\users\asmarus\anaconda3\envs\tensorflow-gpu\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


f1_score =  nan
20 	10    	nan	nan	nan	nan
BEST PARAMETERS =  ['gini', 'best', 43, 0.07250229681880183, 0.08064296231376233, 0.4924053632409452, 0.8307876692534332, 54, 0.10868165602321989, None]


G:\USB Drive Files\Year 3\Semester 1\KV6003 - Individual Computing Project\Machine Learning\kf_ml_lib.py:91: RuntimeWarning: invalid value encountered in double_scalars
  f1_score = 2 * ((precision*recall) / (precision+recall))


([<matplotlib.axis.XTick at 0x179a4f8d688>,
 <a list of 5 Text xticklabel objects>)

In [ ]:
print(y)

y = np.split(y, '')
print(y)